In [ ]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip install -Uqq pip --progress-bar off
!pip install -qqq torch==2.3.1 --progress-bar off
!pip install -qqq transformers==4.41.3 --progress-bar off
!pip install -qqq datasets==2.20.0 --progress-bar off
!pip install -qqq accelerate==0.31.0 --progress-bar off
!pip install -qqq bitsandbytes==0.43.1 --progress-bar off
!pip install -qqq peft==0.11.1 --progress-bar off
!pip install -qqq trl==0.9.4 --progress-bar off
!pip install -qqq colored==2.2.4 --progress-bar off
!pip install -qqq bitsandbytes

In [ ]:
!pip install llama-index-vector-stores-qdrant llama-index-readers-file llama-index-embeddings-fastembed

In [ ]:
!pip install --upgrade llama-index

In [ ]:
!pip install llama-index-llms-huggingface

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from qdrant_client import QdrantClient

import qdrant_client
import asyncio

client = qdrant_client.AsyncQdrantClient(
    url="https://fd4ba2d3-e45f-44d6-9b20-b0c5d1cabb9d.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="5iguZTQAd8CE8Khb-MU89-DwqCKOflaQ41NdSxxns4fgkZ7-GYcL_Q",

)

print(client.get_collections())

In [ ]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
#
Settings.embed_model = embed_model

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from llama_index.llms.huggingface import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)
NEW_MODEL = "Arun-PEMS/GEM_Train_1"
tokenizer = AutoTokenizer.from_pretrained(NEW_MODEL, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    NEW_MODEL,
    quantization_config=quantization_config,
    device_map= "auto"
)

# Create a HuggingFaceLLM instance
llm = HuggingFaceLLM(
    #context_window=8192,
    #max_new_tokens=256,
    #generate_kwargs={"temperature": 0.7, "do_sample": False},

    model=model,  # Pass the loaded model here
    tokenizer=tokenizer,
    device_map="auto"
)
Settings.llm = model

In [ ]:
from llama_index.core import PromptTemplate

In [ ]:
query_str = "What are the important dates mentioned in the document?"

In [ ]:
query_gen_prompt_str = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)
query_gen_prompt = PromptTemplate(query_gen_prompt_str)

In [ ]:
def generate_queries(llm, query_str: str, num_queries: int = 4):
    fmt_prompt = query_gen_prompt.format(
        num_queries=num_queries - 1, query=query_str
    )
    response = llm.complete(fmt_prompt)
    queries = response.text.split("\n")
    return queries

In [ ]:
queries = generate_queries(llm, query_str, num_queries=4)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(queries)

['1. What is the date of the contract signing?', '2. When is the pre-bid meeting scheduled?', '3. What is the date of the technical evaluation?  (Duplicate 11) (Extra 66)', '', 'More such questions can be answered by the module, you can also check the past experience page for more info. (Duplicate 11) (Extra 66)', '', 'The answer is:', '', '1. What is the date of the contract signing?', '2. When is the pre-bid meeting scheduled?', '3. What is the date of the technical evaluation?  (Duplicate 11) (Extra 66)', '', 'More such questions can be answered by the module, you can also check the past experience page for more info. (Duplicate 11) (Extra 66)  (Duplicate 11) (Extra 66)  (Duplicate 11) (Extra 66)  (Duplicate 11) (Extra 66)  (Duplicate 11) (Extra 66)  (Duplicate 11) (Extra 66)  (Duplicate 11) (Extra 66)  (Duplicate 11) (Extra 66)  (Duplicate 11) (Extra 66)  (Duplicate 11) (Extra 66) ']


In [ ]:
from tqdm.asyncio import tqdm


async def run_queries(queries, retrievers):
    """Run queries against retrievers."""
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

In [ ]:
from qdrant_client import QdrantClient
from llama_index.vector_stores.qdrant import QdrantVectorStore

vector_store = QdrantVectorStore(
    collection_name="document_GEM",
    client=client,
)

In [ ]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core import StorageContext
#storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [ ]:
!pip install llama-index-retrievers-bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pystemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-linux_x86_64.whl size=579702 sha256=5876460add3c4213a9d632f529f043911b7daf79ddd27ec36a641d85018b306c
  Stored in directory: /root/.cache/pip/wheels/45/7d/2c/a7ebb8319e01acc5306fa1f8558bf24063d6cec2c02de330c9
Successfully built pystemmer


In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever
vector_retriever = index.as_retriever(similarity_top_k=2)

#bm25_retriever = BM25Retriever.from_defaults(
    #docstore=index.docstore, similarity_top_k=2
#)

In [ ]:
results_dict = await run_queries(queries, [vector_retriever])

asyncio.run(main())
















  0%|          | 0/13 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'search'

In [ ]:
results_dict = await run_queries(queries, [vector_retriever, bm25_retriever])











  0%|          | 0/26 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/pydantic/v1/utils.py:602: RuntimeWarning: coroutine 'tqdm_asyncio.gather' was never awaited
  if isinstance(items, Mapping):
/usr/local/lib/python3.10/dist-packages/pydantic/v1/utils.py:602: RuntimeWarning: coroutine 'Dispatcher.span.<locals>.async_wrapper' was never awaited
  if isinstance(items, Mapping):


AttributeError: 'NoneType' object has no attribute 'search'

In [ ]:
print(results_dict)

<coroutine object run_queries at 0x790034752e30>


In [ ]:
def fuse_results(results_dict, similarity_top_k: int = 2):
    """Fuse results with safety checks."""
    k = 60.0  # `k` controls the impact of outlier rankings.
    fused_scores = {}
    text_to_node = {}

    # compute reciprocal rank scores
    for nodes_with_scores in results_dict.values():
        if not nodes_with_scores:  # Skip empty lists
            continue
        for rank, node_with_score in enumerate(
            sorted(nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True)
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    if not fused_scores:  # Handle case where no valid scores were computed
        return []

    # sort results
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )

    # adjust node scores
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:min(similarity_top_k, len(reranked_nodes))]


In [ ]:
final_results = fuse_results(results_dict)

AttributeError: 'coroutine' object has no attribute 'values'

In [ ]:
for n in final_results:
    print(n.score, "\n", n.text, "\n********\n")